In [4]:
import os
import hashlib

Para implementar a assinatura diigital delithium seguimos o template pois era de uma complexidade notavelmente menor. Fomos capazes de fazer a geração, a assinatura e a verificação corretamente.

Mais uma vez não vamos falar repetir o que foi explicado na documentação e vamos apenas falar um pouco sobre as dúvidas e dificuldades que encontrámos.

Em primeiro lugar vale dizer que não entendemos muito bem o raciocínio por detrás das funções **HighBits**, **LowBits** e **SampleInBall** mas como os algorítmos destas funções estão respresentados no documento conseguimos gerá-las corretamente.

A nossa maior dificuldade foi no entanto na concatenação de w1 e M não por motivos matemáticos mas por conversão de tipos no sage math para possibilitar a sua concatenação. A resposta acabou por ser mais simples que o que estavamos à espera.

In [5]:
class Delithium_Template():
    def __init__(self):
        self.n     = 256
        self.q     = 2 ^ 23 - 2 ^ 13 + 1
        self.k     = 8
        self.l     = 7
        self.eta   = 2
        self.tau   = 60
        self.beta  = 120
        self.gama1 = 2^19
        self.gama2 = (self.q) - 1 / 32
        
        Zx.<x>  = ZZ[]
        Zq.<z>  = PolynomialRing(GF(self.q))
        self.R  = QuotientRing(Zx, x^self.n+1)
        self.Rq = QuotientRing(Zq, z^self.n+1)
        self.Mr = MatrixSpace(self.Rq, self.k, self.l)

        # Helper
    def decompose(self, r, alfa):
        r  = mod(r, self.q)
        r0 = int(mod(r, int(alfa)))
        if (r - r0 == self.q - 1):
            r1 = 0
            r0 = r0 - 1
        else:
            r1 = (r - r0) / int(alfa)
        return (r1, r0)

    def high_bits(self, r, alfa):
        (r1, r0) = self.decompose(r, alfa)
        return r1

    def low_bits(self, r, alfa):
        (r1, r0) = self.decompose(r, alfa)
        return r0

    def high_bits_vector(self, poli, alfa):
        k = poli.list()
        for i in range(len(k)):
            h = k[i]
            h = h.list()
            for j in range(len(h)):
                h[j] = self.high_bits(int(h[j]), alfa)
            k[i]=h
        return k

    def low_bits_vector(self, poli, alfa):
        k = poli.list()
        for i in range(len(k)):
            h = k[i]
            h = h.list()
            for j in range(len(h)):
                h[j] = self.low_bits(int(h[j]), alfa)
            k[i] = h
        return k

    #Passa de Bytes para bits
    def access_bit(self, data, num):                              
        base  = int(num // 8)
        shift = int(num % 8)
        return (data[base] & (1 << shift)) >> shift

    def sample_in_ball(self,  r):
        sl = [self.access_bit(r[:8], i) for i in range(len(r[:8]) * 8)]
        k = 8 # descartar primeiros 8
        c = [0] * 256 

        for i in range (256 - self.tau, 256):
            while (int(r[k]) > i):
                k +=1 
            j = int(r[k])
            k += 1
            s = int(sl[i - 196])
            c[i] = c[j]
            c[j] = (-1) ^ (s)
        return c

    def Shake(self, a, b):
        shake = hashlib.shake_256()
        shake.update(a + b)
        s = shake.digest(int(256))
        return s

    def Hash(self, a, b):
        r = self.Shake(a,b)
        c = self.sample_in_ball(r)
        return c

    def norma_infinito(self, pol):
        J = pol.list()
        for i in range(len(J)):
            k = J[i]
            K = k.list()
            for j in range(len(K)):
                K[j] = abs(int(K[j]))
            J[i] = K
        L = []
        for i in range(len(J)):
            L.append(max(J[i]))
        return max(L)

    def norma_inf_vet(self, vetor):
        for i in range(vetor.nrows()):
            norm = self.norma_infinito(vetor[i])
            vetor[i] = norm
        return max(vetor)

    def norma_inf_matriz(self, matriz):
        L = []
        for i in range(len(matriz)):
            k = matriz[i]
            for j in range(len(k)):
                if k[j] < 0:
                    k[j] = abs(k[j])
                L.append(max(k))
        for i in range(len(L)):
            J = []
            J.append(max(L))
        return J[0]

    # Geração a matriz A em Rq
    def gen_matrix(self):
        matrix = []
        for i in range(self.k * self.l):
            matrix.append(self.Rq.random_element())
        return self.Mr(matrix)
         
    
    #Gerar vetores S em Rq 
    def gen_vector(self, limit, tam):
        matrix = []
        for i in range(tam):
            polinom = []
            for j in range(self.n):
                polinom.append(randint(1, limit))
            matrix.append(self.Rq(polinom))
        return MatrixSpace(self.Rq, tam, 1)(matrix)
        
    
    def keygen(self):
        A = self.gen_matrix()
        s1 = self.gen_vector(self.eta, self.l)
        s2 = self.gen_vector(self.eta, self.k)
        t = A * s1 + s2
        pubkey = (A, t)
        privkey = (A, t, s1, s2)
        
        return pubkey, privkey
    
    def sign(self, seckey, M): 
        A, t, s1, s2 = seckey
        z = 0
        while(z == 0):
            y  = self.gen_vector(self.gama1 - 1 , self.l)
            w1 = str(self.high_bits_vector(A * y, 2 * self.gama2)).encode()
            c  = self.Hash(M.encode(), w1)
            cq = self.Rq(c)
            z  = y + cq * s1
            if self.norma_inf_vet(z)[0] >= self.gama1 - self.beta and self.norma_inf_matriz(self.low_bits_vector(A * y - cq * s2, 2 * self.gama2)) >= self.gama2 - self.beta:
                return z, c
            else:
                z = 0
            
            
    # Função para verificar uma assinatura
    def verify(self,pk, M, sigma):
        A, t = pk
        z, c = sigma
        cq = self.Rq(c)
        w1 = str(self.high_bits_vector(A * z - cq * t, 2 * self.gama2)).encode()
        c1 = self.Hash(M.encode(), w1)
        
        if int(self.norma_inf_vet(z)[0]) < float(self.gama2 - self.beta) and c1 == c:
            print("Assinatura Válida!")
        else:
            print("Assinatura Inválida!")
    

In [6]:
d = Delithium_Template()

msg = 'hello'
pk, sk = d.keygen()
sigma = d.sign(sk, msg)
d.verify(pk, msg, sigma)


Assinatura Válida!
